# Imports

In [47]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_curve, auc, f1_score, classification_report, confusion_matrix, balanced_accuracy_score
import numpy as np
from catboost import CatBoostClassifier, Pool
import matplotlib.pyplot as plt
from utils.feature_utils import generate_features


# Auxiliray Functions

In [24]:
def create_column_summary_table(data_frame: pd.DataFrame):
    """
    Creates a Pandas DataFrame summarizing basic EDA information for each column.

    Args:
        df (pd.DataFrame): Input DataFrame.

    Returns:
        pd.DataFrame: DataFrame with EDA summary (% missing, % unique, n_unique, column type).
    """
    eda_list = []
    for col in data_frame.columns:
        missing_percent = (data_frame[col].isnull().sum() / len(data_frame)) * 100
        unique_percent = (data_frame[col].nunique() / len(data_frame)) * 100
        n_unique = data_frame[col].nunique()
        dtype = data_frame[col].dtype
        eda_list.append([missing_percent, unique_percent, n_unique, dtype])

    eda_table = pd.DataFrame(
        eda_list, 
        columns=['% Missing Values', '% Unique Values', 'N Unique Values', 'Column Type'], 
        index=data_frame.columns
    )
    # Round percentage columns to 2 decimals
    eda_table['% Missing Values'] = eda_table['% Missing Values'].round(2)
    eda_table['% Unique Values'] = eda_table['% Unique Values'].round(2)
    return eda_table

In [26]:
def count_duplictated_rows(data_frame: pd.DataFrame):
    total_duplicates = data_frame.duplicated().sum()
    total_duplicate_percent = (total_duplicates / len(data_frame)) * 100
    print(f"\nTotal duplicate rows: {total_duplicates} ({total_duplicate_percent:.2f}% of dataset)")

    # Display duplicate combinations of customer_id and date
    customer_date_duplicates = data_frame.duplicated(subset=['customer_id', 'date']).sum()
    customer_date_duplicate_percent = (customer_date_duplicates / len(data_frame)) * 100
    print(f"Duplicate user-app combinations: {customer_date_duplicates} ({customer_date_duplicate_percent:.2f}% of dataset)")

In [21]:
def show_df(data_frame: pd.DataFrame):
    print(f'Number of rows: {data_frame.shape[0]}, Number of columns: {data_frame.shape[1]}')
    print(f'Number of duplicated rows: data_frame.duplicated().sum()')
    print("First 2 rows:")
    display(data_frame.head(10))

In [9]:
def find_non_numeric_values(series):
    non_numeric = series[pd.to_numeric(series, errors='coerce').isna() & series.notna()]
    return non_numeric.unique()

In [10]:
def plot_distribution(values: pd.Series, 
                     title: str,
                     log_scale: bool = False, 
                     figsize: tuple = (12, 8),
                     bins: int = 30) -> None:
    """
    Plot distribution and boxplot for a given series of values.
    
    Parameters:
    -----------
    values : pd.Series
        Values to plot
    title : str
        Title for the plot
    log_scale : bool, default=False
        Whether to use log scale for the plots
    figsize : tuple, default=(12, 8)
        Figure size in inches (width, height)
    bins : int, default=30
        Number of bins for the histogram
    """
    # Create figure with stacked subplots
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=figsize, height_ratios=[2, 1])
    
    # Plot histogram
    sns.histplot(values, bins=bins, ax=ax1)
    ax1.set_title(f'Distribution of {title}')
    ax1.set_xlabel('')
    ax1.set_ylabel('Count')
    
    if log_scale:
        ax1.set_yscale('log')
    
    # Plot boxplot
    sns.boxplot(x=values, ax=ax2)
    ax2.set_xlabel(title)
    
    if log_scale:
        ax2.set_xscale('log')
    
    plt.tight_layout()
    plt.show()

In [11]:
def plot_permission_distribution(data_frame: pd.DataFrame):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

    # First subplot - Permissions by Seniority
    perm_by_seniority = data_frame.groupby('seniority_inverted', observed=True)['permission'].value_counts(normalize=True).unstack() * 100
    perm_by_seniority.plot(kind='bar', width=0.8, ax=ax1)

    ax1.set_title('Permission Distribution by Seniority Level')
    ax1.set_xlabel('Seniority Level')
    ax1.set_ylabel('Percentage')
    ax1.legend(title='Permission', labels=['Denied (0)', 'Granted (1)'])

    # Add percentage labels on the bars for first subplot
    for container in ax1.containers:
        ax1.bar_label(container, 
                    label_type='center',
                    fmt='%.1f%%',
                    color='white',
                    fontweight='bold')

    # Second subplot - Permissions by isMachine
    perm_by_machine = data_frame.groupby('isMachine', observed=True)['permission'].value_counts(normalize=True).unstack() * 100
    perm_by_machine.plot(kind='bar', width=0.8, ax=ax2)

    ax2.set_title('Permission Distribution by Machine Status')
    ax2.set_xlabel('Is Machine')
    ax2.set_ylabel('Percentage')
    ax2.legend(title='Permission', labels=['Denied (0)', 'Granted (1)'])

    # Add percentage labels on the bars for second subplot
    for container in ax2.containers:
        ax2.bar_label(container, 
                    label_type='center',
                    fmt='%.1f%%',
                    color='white',
                    fontweight='bold')

    # Adjust x-axis labels for isMachine
    ax2.set_xticklabels(['Human (0)', 'Machine (1)'])

    plt.tight_layout()
    plt.show()


In [12]:
def plot_is_machine_seniority_distribution(data_frame: pd.DataFrame):
    machine_by_seniority = data_frame.groupby('seniority_inverted', observed=True)['isMachine'].value_counts(normalize=True).unstack() * 100

    # Create the plot
    plt.figure(figsize=(10, 6))
    ax = machine_by_seniority.plot(kind='bar', width=0.8)

    plt.title('Distribution of Machine vs. Human Users by Seniority Level')
    plt.xlabel('Seniority Level')
    plt.ylabel('Percentage')
    plt.legend(title='Is Machine', labels=['Human (0)', 'Machine (1)'])

    # Add percentage labels on the bars
    for container in ax.containers:
        ax.bar_label(container, 
                    label_type='center',
                    fmt='%.1f%%',
                    color='white',
                    fontweight='bold')

    plt.tight_layout()
    plt.show()

In [13]:
def plot_permission_by_dep_cat_distributions(data_frame: pd.DataFrame):
    # Create figure with a 2x2 grid
    fig = plt.figure(figsize=(20, 16))
    gs = fig.add_gridspec(2, 2)
    
    # Create four axes
    ax1 = fig.add_subplot(gs[0, 0])  # top left
    ax2 = fig.add_subplot(gs[0, 1])  # top right
    ax3 = fig.add_subplot(gs[1, 0])  # bottom left
    ax4 = fig.add_subplot(gs[1, 1])  # bottom right
    
    # Plot 1: Categories (top left)
    perm_by_category = data_frame.groupby('category', observed=False)['permission'].value_counts(normalize=True).unstack() * 100
    perm_by_category = perm_by_category.sort_values(by=1, ascending=True)
    perm_by_category.plot(kind='barh', width=0.8, ax=ax1)
    ax1.set_title('Permission Distribution by Category')
    ax1.set_xlabel('Percentage')
    ax1.set_ylabel('Category')
    ax1.legend(title='Permission', labels=['Denied (0)', 'Granted (1)'])
    
    # Plot 2: Departments (top right)
    perm_by_dept = data_frame.groupby('department', observed=False)['permission'].value_counts(normalize=True).unstack() * 100
    perm_by_dept = perm_by_dept.sort_values(by=1, ascending=True)
    perm_by_dept.plot(kind='barh', width=0.8, ax=ax2)
    ax2.set_title('Permission Distribution by Department')
    ax2.set_xlabel('Percentage')
    ax2.set_ylabel('Department')
    ax2.legend(title='Permission', labels=['Denied (0)', 'Granted (1)'])
    
    # Plot 3: Office Locations (bottom left)
    perm_by_location = data_frame.groupby('officeLocation', observed=False)['permission'].value_counts(normalize=True).unstack() * 100
    perm_by_location = perm_by_location.sort_values(by=1, ascending=True)
    perm_by_location.plot(kind='barh', width=0.8, ax=ax3)
    ax3.set_title('Permission Distribution by Office Location')
    ax3.set_xlabel('Percentage')
    ax3.set_ylabel('Office Location')
    ax3.legend(title='Permission', labels=['Denied (0)', 'Granted (1)'])
    
    # Plot 4: Permission Rates by App (bottom right)
    app_stats = data_frame.groupby('appId', observed=False)['permission'].agg(['mean', 'count'])
    app_stats = app_stats.sort_values('mean', ascending=True)
    
    # Plot permission rates
    app_stats['mean'].plot(kind='hist', bins=30, ax=ax4)
    ax4.set_title('Distribution of Permission Rates Across Apps')
    ax4.set_xlabel('Permission Rate')
    ax4.set_ylabel('Count of Apps')
    
    # Add mean and median lines
    mean_rate = app_stats['mean'].mean()
    median_rate = app_stats['mean'].median()
    
    # Add mean line (red)
    ax4.axvline(x=mean_rate, color='r', linestyle='--', alpha=0.7, label=f'Mean: {mean_rate:.1%}')
    
    # Add median line (blue)
    ax4.axvline(x=median_rate, color='b', linestyle='--', alpha=0.7, label=f'Median: {median_rate:.1%}')
    
    # Add legend for mean and median lines
    ax4.legend()
    
    # Add percentage labels on bars for first three plots
    for ax in [ax1, ax2, ax3]:
        for container in ax.containers:
            ax.bar_label(container, 
                        label_type='center',
                        fmt='%.1f%%',
                        color='white',
                        fontweight='bold')
    
    plt.tight_layout()
    plt.show()

In [14]:
def plot_permission_rate_and_isMachine_dist(data_frame: pd.DataFrame):
    # Create figure with two subplots
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

    # Plot 1: Overall Permission Distribution
    permission_dist = data_frame['permission'].value_counts(normalize=True) * 100
    permission_dist.plot(kind='bar', ax=ax1)
    ax1.set_title('Overall Permission Distribution')
    ax1.set_xlabel('Permission')
    ax1.set_ylabel('Percentage')
    ax1.set_xticklabels(['Denied (0)', 'Granted (1)'])

    # Add percentage labels on bars
    for i, v in enumerate(permission_dist):
        ax1.text(i, v/2, f'{v:.1f}%', ha='center', color='white', fontweight='bold')

    # Plot 2: isMachine Distribution
    machine_dist = data_frame['isMachine'].value_counts(normalize=True) * 100
    machine_dist.plot(kind='bar', ax=ax2)
    ax2.set_title('Machine vs Human Distribution')
    ax2.set_xlabel('Is Machine')
    ax2.set_ylabel('Percentage')
    ax2.set_xticklabels(['Human (0)', 'Machine (1)'])

    # Add percentage labels with adjusted position for small values
    for i, v in enumerate(machine_dist):
        if v < 10:  # For small values, place text above the bar
            ax2.text(i, v + 1, f'{v:.1f}%', ha='center', color='black', fontweight='bold')
        else:  # For larger values, place text in the middle of the bar
            ax2.text(i, v/2, f'{v:.1f}%', ha='center', color='white', fontweight='bold')

    # Adjust y-axis to make room for labels above small bars
    ax2.set_ylim(0, max(machine_dist) * 1.1)  # Add 10% padding at the top

    plt.tight_layout()
    plt.show()

In [15]:
def plot_seniority_distribution(data_frame: pd.DataFrame):
    # Create figure
    plt.figure(figsize=(10, 6))

    # Calculate and sort seniority distribution
    seniority_dist = data_frame['seniority_inverted'].value_counts(normalize=True) * 100
    seniority_dist = seniority_dist.sort_index()  # Sort by seniority level

    # Create bar plot
    ax = seniority_dist.plot(kind='bar')
    plt.title('Seniority Level Distribution')
    plt.xlabel('Seniority Level')
    plt.ylabel('Percentage')

    # Add percentage labels with adjusted position for small values
    for i, v in enumerate(seniority_dist):
        if v < 10:  # For small values, place text above the bar
            plt.text(i, v + 0.5, f'{v:.1f}%', ha='center', color='black', fontweight='bold')
        else:  # For larger values, place text in the middle of the bar
            plt.text(i, v/2, f'{v:.1f}%', ha='center', color='white', fontweight='bold')

    # Adjust y-axis to make room for labels above small bars
    plt.ylim(0, max(seniority_dist) * 1.1)  # Add 10% padding at the top

    plt.tight_layout()
    plt.show()

In [16]:
def bin_department(data_frame: pd.DataFrame):
    dept_permission_rates = data_frame.groupby('department', observed=True)['permission'].mean().reset_index()
    dept_permission_rates.columns = ['department', 'grant_rate']

    n_dept_bins = 5
    dept_permission_rates['dept_bin'] = pd.qcut(
        dept_permission_rates['grant_rate'], 
        n_dept_bins, 
        labels=[f'dept_bin_{i+1}' for i in range(n_dept_bins)]
    )

    dept_to_bin = dict(zip(dept_permission_rates['department'], dept_permission_rates['dept_bin']))

    data_frame['department_binned'] = data_frame['department'].map(dept_to_bin)

    return data_frame

In [17]:
def bin_office_location(data_frame: pd.DataFrame):
    loc_permission_rates = data_frame.groupby('officeLocation', observed=True)['permission'].mean().reset_index()
    loc_permission_rates.columns = ['officeLocation', 'grant_rate']

    n_loc_bins = 5
    loc_permission_rates['loc_bin'] = pd.qcut(
        loc_permission_rates['grant_rate'], 
        n_loc_bins, 
        labels=[f'loc_bin_{i+1}' for i in range(n_loc_bins)]
    )

    loc_to_bin = dict(zip(loc_permission_rates['officeLocation'], loc_permission_rates['loc_bin']))

    # Step 8: Apply location binning to original dataframe
    data_frame['officeLocation_binned'] = data_frame['officeLocation'].map(loc_to_bin)

    return data_frame

In [18]:
def train_and_evaluate(df_train, df_test, categorical_features, numeric_features, model_path=None):
    """Train CatBoost model with CV and evaluate on test set."""
    print("Feature Information:")
    print(f"Categorical features: {categorical_features}")
    print(f"Numeric features: {numeric_features}")
    
    # Calculate class weights
    class_counts = df_train['permission'].value_counts()
    n_samples = len(df_train)
    n_classes = len(class_counts)
    class_weights = {i: n_samples / (n_classes * count) for i, count in class_counts.items()}
    
    # Good default parameters for CatBoost
    params = {
        'iterations': 500,
        'depth': 6,
        'learning_rate': 0.1,
        'l2_leaf_reg': 3,
        'min_data_in_leaf': 20,
        'max_bin': 200,
        'random_strength': 1,
        'bagging_temperature': 1,
        'class_weights': class_weights,
        'thread_count': -1,  # Use all CPU cores
        'task_type': 'GPU',  # Enable GPU training
        'devices': '0',      # Use first GPU device
        'verbose': False,
        'gpu_ram_part': 0.95,     # Use 95% of GPU memory
        'pinned_memory_size': '1gb',
        'class_names': [0, 1],
    }
    
    # Prepare features
    X = df_train[categorical_features + numeric_features]
    y = df_train['permission']
    
    # Create stratification column
    df_train['strat'] = df_train['permission'].astype(str) + '_' + df_train['isMachine'].astype(str)
    
    # Perform cross-validation
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    cv_scores = []
    
    print("\nPerforming 5-fold CV...")
    for fold, (train_idx, val_idx) in enumerate(cv.split(X, df_train['strat']), 1):
        X_fold_train, X_fold_val = X.iloc[train_idx], X.iloc[val_idx]
        y_fold_train, y_fold_val = y.iloc[train_idx], y.iloc[val_idx]
        
        # Create pools
        train_pool = Pool(
            X_fold_train,
            y_fold_train,
            cat_features=categorical_features
        )
        val_pool = Pool(
            X_fold_val,
            y_fold_val,
            cat_features=categorical_features
        )
        
        # Train model
        model = CatBoostClassifier(**params, random_seed=42)
        model.fit(train_pool, use_best_model=True)
        
        # Evaluate
        y_pred = model.predict(val_pool)
        fold_score = f1_score(y_fold_val, y_pred)
        cv_scores.append(fold_score)
        print(f"Fold {fold} F1 Score: {fold_score:.4f}")
    
    print(f"\nMean CV F1 Score: {np.mean(cv_scores):.4f} (+/- {np.std(cv_scores):.4f})")
    
    # Train final model on full training data
    print("\nTraining final model on full training data...")
    final_model = CatBoostClassifier(**params, random_seed=42)
    
    train_pool = Pool(
        X,
        y,
        cat_features=categorical_features
    )
    
    final_model.fit(train_pool)
    
    # Feature importance
    feature_importance = pd.DataFrame({
        'Feature': final_model.feature_names_,
        'Importance': final_model.get_feature_importance()
    }).sort_values('Importance', ascending=False)
    print("\nTop 10 Feature Importance:")
    print(feature_importance.head(10))
    
    # Evaluate on test set
    print("\nTest Set Performance:")
    X_test = df_test[categorical_features + numeric_features]
    y_test = df_test['permission']
    
    test_pool = Pool(
        X_test,
        y_test,
        cat_features=categorical_features
    )
    
    y_pred = final_model.predict(test_pool)
    y_pred_proba = final_model.predict_proba(test_pool)[:, 1]
    
    fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
    roc_auc = auc(fpr, tpr)

    # if roc_auc < 0.5:
    #     y_pred_proba = 1 - y_pred_proba
    #     fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
    #     roc_auc = auc(fpr, tpr)
    
    # Store all results
    test_results = {
        'f1_score': f1_score(y_test, y_pred),
        'classification_report': classification_report(y_test, y_pred),
        'confusion_matrix': confusion_matrix(y_test, y_pred),
        'roc_curve': {
            'fpr': fpr,
            'tpr': tpr
        },
        'auc_score': roc_auc,
        'y_pred_proba': y_pred_proba,
        'accuracy': balanced_accuracy_score(y_test, y_pred)
    }
    
    # Print results nicely
    print_results(test_results)
    
    # Save model if path is provided
    if model_path:
        final_model.save_model(model_path)
        print(f"\nModel saved to: {model_path}")
    
    return final_model, test_results

def print_results(results):
    """Print classification results in a formatted way."""
    print("\n" + "="*50)
    print("MODEL PERFORMANCE METRICS")
    print("="*50)
    
    # F1 Score and AUC Score
    print(f"\nF1 Score: {results['f1_score']:.4f}")
    print(f"AUC Score: {results['auc_score']:.4f}")
    
    # Classification Report
    print("\nDetailed Performance Metrics:")
    print("-"*50)
    print(results['classification_report'])
    
    # Confusion Matrix
    cm = results['confusion_matrix']
    print("Confusion Matrix:")
    print("-"*50)
    print(f"                Predicted NO  Predicted YES")
    print(f"Actual NO     {cm[0,0]:>11,d} {cm[0,1]:>13,d}")
    print(f"Actual YES    {cm[1,0]:>11,d} {cm[1,1]:>13,d}")
    
    # Additional metrics
    total = cm.sum()
    tn, fp, fn, tp = cm.ravel()
    
    print("\nAdditional Metrics:")
    print("-"*50)
    print(f"Total Samples:    {total:,d}")
    print(f"Correct:          {(tn + tp):,d}")
    print(f"Incorrect:        {(fp + fn):,d}")
    print(f"Accuracy:         {(tn + tp) / total:.4f}")
    print(f"Misclass. Rate:   {(fp + fn) / total:.4f}")
    print(f"Precision:        {tp / (tp + fp):.4f}")
    print(f"Recall:          {tp / (tp + fn):.4f}")
    print("="*50)
    
    # Plot ROC curve

    plt.figure(figsize=(8, 6))
    plt.plot(results['roc_curve']['fpr'], results['roc_curve']['tpr'], 
             color='darkorange', lw=2,
             label=f'ROC curve (AUC = {results["auc_score"]:.4f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend(loc="lower right")
    plt.grid(True)
    plt.show()

# Section 1 - Data Inspection

### Loading data

In [62]:
df = pd.read_csv('data/raw/churn_data.csv')

### data inspection

In [63]:
show_df(df)

Number of rows: 1200, Number of columns: 6
Number of duplicated rows: data_frame.duplicated().sum()
First 2 rows:


,customer_id,date,transaction_amount,plan_type,churn,issuing_date
0,CUST_1,2023-01-01,193.524658,Basic,0,2021-03-01
1,CUST_1,2023-02-01,303.342657,Standard,0,2021-03-01
2,CUST_1,2023-03-01,38.460970,Standard,0,2021-03-01
3,CUST_1,2023-04-01,356.955563,Premium,0,2021-03-01
4,CUST_1,2023-05-01,417.896894,Standard,0,2021-03-01
5,CUST_1,2023-06-01,NaN,Premium,0,2021-03-01
6,CUST_1,2023-07-01,221.653059,Standard,0,2021-03-01
7,CUST_1,2023-08-01,78.351992,Standard,0,2021-03-01
8,CUST_1,2023-09-01,233.474292,Premium,0,2021-03-01
9,CUST_1,2023-10-01,261.974875,Standard,1,2021-03-01


## Merged data frame inspection

In [64]:
print(create_column_summary_table(df))
print(count_duplictated_rows(df))

                    % Missing Values  % Unique Values  N Unique Values  \
customer_id                     0.00             8.33              100   
date                            0.00             1.00               12   
transaction_amount              0.58            99.42             1193   
plan_type                       0.08             0.25                3   
churn                           0.00             0.17                2   
issuing_date                    0.00             4.42               53   

                   Column Type  
customer_id             object  
date                    object  
transaction_amount     float64  
plan_type               object  
churn                    int64  
issuing_date            object  

Total duplicate rows: 0 (0.00% of dataset)
Duplicate user-app combinations: 0 (0.00% of dataset)
None


In [69]:
def find_non_consecutive_churn(df):
    """
    Efficiently identifies customers with non-consecutive churn patterns.
    Returns a list of customer IDs where churn goes from 1 back to 0.
    """
    # Create a copy of the dataframe with only necessary columns
    temp_df = df[['customer_id', 'date', 'churn']].copy()
    
    # Convert date to datetime and sort
    temp_df['date'] = pd.to_datetime(temp_df['date'])
    temp_df = temp_df.sort_values(['customer_id', 'date'])
    
    # Create a shifted column to compare consecutive values
    temp_df['prev_churn'] = temp_df.groupby('customer_id')['churn'].shift(1)
    
    # Find rows where churn went from 1 to 0
    invalid_transitions = temp_df[(temp_df['prev_churn'] == 1) & (temp_df['churn'] == 0)]
    
    # Get unique customer IDs with invalid transitions
    non_consecutive_customers = invalid_transitions['customer_id'].unique().tolist()
    
    print(f"Found {len(non_consecutive_customers)} customers with non-consecutive churn patterns")
    
    return non_consecutive_customers

In [70]:
non_consecutive_customers = find_non_consecutive_churn(df)

Found 0 customers with non-consecutive churn patterns


In [76]:
df[df.plan_type.isna()]

,customer_id,date,transaction_amount,plan_type,churn,issuing_date
948,CUST_80,2023-01-01,145.323083,NaN,0,2021-12-01


In [75]:
df.groupby('customer_id').size().to_frame()[0].unique()


array([12], dtype=int64)

In [55]:
tmp = generate_features(df)

last_plan_type: The plan type in the most recent month (December 2023)
is_churned_last_month: Whether the customer was in churned state in December 2023
tenure_months: Number of months the customer has been with the service as of December 2023 ---
days_since_plan_change: Number of days since the customer last changed their plan
mom_transaction_change: Month-over-month percentage change in transaction amount (Nov to Dec) ---- less
recent_volatility: Standard deviation of transaction amounts in the last 3 months ---- can catch neg' or pos'?
avg_transaction_amount: Average transaction amount across all months ---- how predictive?
transaction_trend: Slope coefficient of transaction amounts over time (increasing/decreasing) ---- can be important
total_plan_changes: Total number of times the customer changed plans during the year
pct_basic_plan: Percentage of months the customer was on the Basic plan
pct_standard_plan: Percentage of months the customer was on the Standard plan
pct_premium_plan: Percentage of months the customer was on the Premium plan
last_plan_change_type: Direction of the most recent plan change (1=upgrade, -1=downgrade, 0=no change)
missing_transaction_months: Number of months with missing transaction data
transaction_cv: Coefficient of variation of transaction amounts (std/mean)
max_transaction_amount: Maximum transaction amount recorded ---- less
min_transaction_amount: Minimum transaction amount recorded ---- less
q1_avg_transaction: Average transaction amount in Q1 (Jan-Mar)
q2_avg_transaction: Average transaction amount in Q2 (Apr-Jun)
q3_avg_transaction: Average transaction amount in Q3 (Jul-Sep)
q4_avg_transaction: Average transaction amount in Q4 (Oct-Dec)
recent_to_historical_ratio: Ratio of recent 6 months' spending to earlier 6 months

tenure_months
transaction_trend
pct_basic_plan
pct_standard_plan
pct_premium_plan
last_plan_change_type
missing_transaction_months
transaction_cv
q3_avg_transaction
q4_avg_transaction

tenure_months
transaction_trend
mom_transaction_change
recent_volatility
last_plan_change_type
total_plan_changes
days_since_plan_change
transaction_cv
recent_to_historical_ratio
pct_premium_plan

In [77]:
tmp.query('customer_id=="CUST_80"').iloc[0]

customer_id                      CUST_80
last_plan_type                  Standard
is_churned_last_month                  0
tenure_months                  25.333333
days_since_plan_change                60
mom_transaction_change         -0.592473
recent_volatility             124.746267
avg_transaction_amount        291.655959
transaction_trend               3.650623
total_plan_changes                     9
pct_basic_plan                  0.333333
pct_standard_plan               0.333333
pct_premium_plan                    0.25
last_plan_change_type                  0
missing_transaction_months             0
transaction_cv                  0.400841
max_transaction_amount        482.398424
min_transaction_amount        145.323083
q1_avg_transaction            313.213977
q2_avg_transaction            190.875212
q3_avg_transaction             363.08576
q4_avg_transaction            299.448888
recent_to_historical_ratio       1.31432
Name: 79, dtype: object

In [78]:
df.query('customer_id=="CUST_80"').sort_values('date')

,customer_id,date,transaction_amount,plan_type,churn,issuing_date
948,CUST_80,2023-01-01,145.323083,NaN,0,2021-12-01
949,CUST_80,2023-02-01,421.275861,Basic,0,2021-12-01
950,CUST_80,2023-03-01,373.042988,Standard,0,2021-12-01
951,CUST_80,2023-04-01,186.514620,Basic,0,2021-12-01
952,CUST_80,2023-05-01,163.540480,Standard,0,2021-12-01
953,CUST_80,2023-06-01,222.570536,Premium,0,2021-12-01
954,CUST_80,2023-07-01,482.398424,Basic,0,2021-12-01
955,CUST_80,2023-08-01,251.968880,Basic,0,2021-12-01
956,CUST_80,2023-09-01,354.889975,Premium,0,2021-12-01
957,CUST_80,2023-10-01,359.392565,Premium,0,2021-12-01
